In [1]:
from email.mime import audio
import numpy as np
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials
import tqdm
import pandas as pd
import shutil

In [143]:
df1 = pd.read_csv('./archive/DEAM_Annotations/annotations/annotations averaged per song/song_level/static_annotations_averaged_songs_1_2000.csv')
df2 = pd.read_csv('./archive/DEAM_Annotations/annotations/annotations averaged per song/song_level/static_annotations_averaged_songs_2000_2058.csv')[['song_id', ' valence_mean', ' valence_std',' arousal_mean',' arousal_std']]

In [147]:
def va_to_cat(r):
    if r[' valence_mean']>5: 
        return 'High Valence'
    else:
        return 'Low Valence'

In [148]:
def ar_to_cat(r):
    if r[' arousal_mean']>5: 
        return 'High Arousal'
    else:
        return 'Low Arousal'

In [149]:
df1['valence_category'] = df1.apply(lambda r: va_to_cat(r),axis = 1)
df2['valence_category'] = df2.apply(lambda r: va_to_cat(r),axis = 1)

In [150]:
df1['arousal_category'] = df1.apply(lambda r: ar_to_cat(r),axis = 1)
df2['arousal_category'] = df2.apply(lambda r: ar_to_cat(r),axis = 1)

In [151]:
df1.arousal_category.value_counts()

Low Arousal     410
High Arousal    341
Name: arousal_category, dtype: int64

In [152]:
df_hvha = df1[(df1.arousal_category == 'High Arousal') & (df1.valence_category == 'High Valence')]
df_hvla = df1[(df1.arousal_category == 'Low Arousal') & (df1.valence_category == 'High Valence')]
df_lvha = df1[(df1.arousal_category == 'High Arousal') & (df1.valence_category == 'Low Valence')]
df_lvla = df1[(df1.arousal_category == 'Low Arousal') & (df1.valence_category == 'Low Valence')]

In [153]:
idx_hvha = df_hvha.song_id.to_numpy()
idx_hvla = df_hvla.song_id.to_numpy()
idx_lvla = df_lvla.song_id.to_numpy()
idx_lvha = df_lvha.song_id.to_numpy()

In [154]:
np.random.shuffle(idx_hvha)
np.random.shuffle(idx_hvla)
np.random.shuffle(idx_lvha)
np.random.shuffle(idx_lvla)

In [155]:
selected = np.concatenate([idx_hvha[:10],idx_hvla[:10],idx_lvha[:10],idx_lvla[:10]])

In [156]:
print(selected)

[ 320  431 1806 1742 1895 1214  302 1216 1849  355 1934 1477 1402  122
 1310  726 1579 1288  729   19  597 1247 1636  668 1991  628 1556  489
   41 1547  116  499  341  234 1602 1056 1407   13  521  301]


In [157]:
df2_new = pd.concat([df2,df1[df1.song_id.isin(selected)]])
df1_new = df1[~df1.song_id.isin(selected)]

In [158]:
df2_new.columns

Index(['song_id', ' valence_mean', ' valence_std', ' arousal_mean',
       ' arousal_std', 'valence_category', 'arousal_category'],
      dtype='object')

In [133]:
df1_new.arousal_category.value_counts()

Low Arousal     390
High Arousal    321
Name: arousal_category, dtype: int64

In [135]:
df2_new.arousal_category.value_counts()

Low Arousal     41
High Arousal    34
Name: arousal_category, dtype: int64

In [159]:
df2_new = df2_new.sort_values(by='song_id')

In [160]:
df1_new.to_csv('./medieval_vocal_train.csv',index = False)
df2_new.to_csv('./medieval_vocal_test.csv',index = False)

In [2]:
data = pd.read_csv('./medieval_vocal_train.csv')
data2 = pd.read_csv('./medieval_vocal_test.csv')